In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import json

transactions = []     # Define a global list to store transactions
category_type = ""    #Creating global categorical type to store user base type

def add_transaction():
    date = input("Enter date (YYYY-MM-DD): ")
    category = input("Enter category: ")
    description = input("Enter description: ")
    amount = float(input("Enter amount: "))
    type_ = input("Enter type (Income/Expense): ")
    transaction = {
        "date": date,
        "category": category,
        "description": description,
        "amount": amount,
        "type": type_
    }
    transactions.append(transaction)
    print("Transaction has been succesfully added")

def edit_transaction():
    index = int(input("Enter transaction index to edit: "))
    if 0 <= index < len(transactions):
        date = input("Enter new date (YYYY-MM-DD) or press Enter to keep current: ")
        category = input("Enter new category or press Enter to keep current: ")
        description = input("Enter new description or press Enter to keep current: ")
        amount = input("Enter new amount or press Enter to keep current: ")
        type_ = input("Enter new type (Income/Expense) or press Enter to keep current: ")

        if date:
            transactions[index]['date'] = date
        if category:
            transactions[index]['category'] = category
        if description:
            transactions[index]['description'] = description
        if amount:
            transactions[index]['amount'] = float(amount)
        if type_:
            transactions[index]['type'] = type_
    else:
        print("Invalid index!")

def delete_transaction():
    index = int(input("Enter transaction index to delete: "))
    if 0 <= index < len(transactions):
        transactions.pop(index)
        print("Transactions has been successfully deleted") 
    else:
        print("Invalid index!")
        
def save_transactions_json(filename='transactions.json'):
    with open(filename, 'w') as file:
        json.dump(transactions, file)
    print(f"Transactions saved to {filename}")

def load_transactions_json(filename='transactions.json'):
    global transactions
    try:
        with open(filename, 'r') as file:
            transactions = json.load(file)
        print(f"Transactions loaded from {filename}")
    except FileNotFoundError:
        print(f"File {filename} not found. Starting with an empty transaction list.")
    
def monthly_summary():
    df = pd.DataFrame(transactions)
    if 'date' not in df.columns:
        print("Error: No 'date' field found in transactions")
        return pd.DataFrame()
    df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d', errors='coerce')  # Ensure correct parsing
    df.dropna(subset=['date'], inplace=True)  # Drop rows where date conversion failed
    df['month'] = df['date'].dt.to_period('M')
    summary = df.groupby(['month', 'type'])['amount'].sum().unstack().fillna(0)
    return summary

def category_summary():
    global category_type
    df = pd.DataFrame(transactions)
    if 'type' not in df.columns or 'category' not in df.columns:
        print("Error: No 'type' or 'category' field found in transactions")
        return pd.DataFrame()
    classification = input("Enter Which Type of Category Summary you want Income/Expense")
    category_type = classification
    summary = df[df['type'] == classification].groupby('category')['amount'].sum()
    return summary

def visualize_monthly_summary():
    summary = monthly_summary()
    if not summary.empty:
        summary.plot(kind='bar', figsize=(10, 6))
        plt.title('Monthly Income vs. Expenses')
        plt.xlabel('Month')
        plt.ylabel('Amount')
        plt.xticks(rotation=45)
        plt.legend(['Expense', 'Income'])
        plt.show()

def visualize_category_summary():
    summary = category_summary()
    if not summary.empty:
        summary.plot.pie(y='amount', autopct='%1.1f%%', figsize=(8, 8), legend=False)
        plt.title(category_type+' by Category')
        plt.ylabel('')
        plt.show()

def display_transactions():
    for i, transaction in enumerate(transactions):
        print(f"{i}: {transaction}")

def main():
    while True:
        print("\nFinance Tracker Menu:")
        print("1.  Add Transaction")
        print("2.  Edit Transaction")
        print("3.  Delete Transaction")
        print("4.  Display Transactions")
        print("5.  Monthly Summary")
        print("6.  Category Summary")
        print("7.  Visualize Monthly Summary")
        print("8.  Visualize Category Summary")
        print("9.  Save in Json Format")
        print("10. Load in Json Format")
        print("11. Exit")
        choice = input("Enter your choice: ")

        if choice == '1':
            add_transaction()
        elif choice == '2':
            edit_transaction()
        elif choice == '3':
            delete_transaction()
        elif choice == '4':
            display_transactions()
        elif choice == '5':
            print(monthly_summary())
        elif choice == '6':
            print(category_summary())
        elif choice == '7':
            visualize_monthly_summary()
        elif choice == '8':
            visualize_category_summary()
        elif choice == '9':
            save_transactions_json()
        elif choice == '10':
            load_transactions_json()
        elif choice == '11':
            print("Thank you For using it!")
            print("Have a great day!")
            break
        else:
            print("Invalid choice! Please try again.")
if __name__ == "__main__":
    main()